In [138]:
# load entities
import json

path = '../data/'

with open(path+'entities.json', 'r') as fin:
    entities = json.load(fin)
print(len(entities), 'entity labels')

28497 entity labels


In [137]:
# load original question
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

path = '../data/'
with open(path+'annotated_wd_data_valid_answerable.txt') as fin:
    lines = fin.readlines()
    l = lines[1]
#         subject [tab] property [tab] object [tab] question
    s, p, o, q = l.strip('\n').split('\t')
    q = re.sub(chars_to_ignore_regex, '', q).lower()
    print(q)
    s_label = entities[s]
    s_label = re.sub(chars_to_ignore_regex, '', s_label).upper()
    print(s_label)

what is a film directed by wiebke von carolsfeld
WIEBKE CAROLSFELD


In [66]:
# match entity label to the ctc table
import os
import torch
from transformers import Wav2Vec2ForCTC
from datasets import load_dataset
import soundfile as sf

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

path = "../data/dev/"

file = '2.wav'
speech, samplerate = sf.read(path+file)
i = int(file.split('.')[0]) - 1

input_values = tokenizer(speech, return_tensors="pt", padding="longest").input_values
logits = model(input_values).logits

# find where s_tokens appear in the table
ctc_table = torch.topk(logits, k=3, dim=-1)
predicted_ids = ctc_table.indices[0]
# predicted_ids = torch.argmax(logits, dim=-1).indices

print(predicted_ids)
print(predicted_ids.shape)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


tensor([[ 0,  6, 10],
        [ 0,  6, 10],
        [ 0,  6, 10],
        ...,
        [ 0,  6,  4],
        [ 0,  6,  4],
        [ 0,  6,  4]])
torch.Size([365, 3])


In [155]:
# encode entity label
from transformers import Wav2Vec2Processor

tokenizer = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

s_tokens = []
for word in s_label.split():
    print(tokenizer.tokenizer._tokenize(word))
    s_tokens.append(tokenizer.tokenizer(word)['input_ids'])

print(s_tokens)

['W', 'I', 'E', 'B', 'K', 'E']
['C', 'A', 'R', 'O', 'L', 'S', 'F', 'E', 'L', 'D']
[[18, 10, 5, 24, 26, 5], [19, 7, 13, 8, 15, 12, 20, 5, 15, 14]]


In [179]:
# produce a score for the label
length = 4
for word in s_tokens:
    print(word)
    score = 0
    ind = 0
    for token_id in word:
#         print(token_id)
        matched_positions = (predicted_ids == torch.Tensor([token_id])).nonzero(as_tuple=True)
        predicted_values = ctc_table.values[0][matched_positions[0], matched_positions[1]]
        # sort positions by logits
        logits, indices = torch.sort(predicted_values, descending=True)
#         print(matched_positions[0][indices][:4])
#         print(matched_positions[1][indices][:4])
#         print(logits[:4])
        
        # get the cumulative probability of the next letter appearing next
        predicted_next = (matched_positions[0][indices][:4] > torch.Tensor([ind])).nonzero(as_tuple=True)[0]
#         print(predicted_next)
        if predicted_next.nelement() != 0:
            cum_p = sum(logits[predicted_next])
#             print(predicted_next[0])
            ind = matched_positions[0][indices][predicted_next[0]]
#             print(ind)
            score += cum_p

    print(score/len(word))

[18, 10, 5, 24, 26, 5]
tensor(30.9749, grad_fn=<DivBackward0>)
[19, 7, 13, 8, 15, 12, 20, 5, 15, 14]
tensor(11.3687, grad_fn=<DivBackward0>)


In [161]:
# produce a score for the label
ind = 0
length = 4
for word in s_tokens:
    print(word)
    score = 0
    for token_id in word:
        print(token_id)
        matched_positions = (predicted_ids == torch.Tensor([token_id])).nonzero(as_tuple=True)
        predicted_values = ctc_table.values[0][matched_positions[0], matched_positions[1]]
        # sort positions by logits
        logits, indices = torch.sort(predicted_values, descending=True)
        print(matched_positions[0][indices][:4])
        print(matched_positions[1][indices][:4])
        print(logits[:4])
        
        # get the cumulative probability of the next letter appearing next
        if matched_positions[0][indices][0] > ind:
            ind = matched_positions[0][indices][0]
            score += logits[0]

    print(score/len(word))

[18, 10, 5, 24, 26, 5]
18
tensor([ 70,  69, 152, 219])
tensor([0, 1, 1, 2])
tensor([13.7542,  9.9319,  6.7443,  5.8215], grad_fn=<SliceBackward>)
10
tensor([ 82, 116, 156, 223])
tensor([0, 0, 0, 2])
tensor([12.0832, 11.4323,  8.8787,  7.1710], grad_fn=<SliceBackward>)
5
tensor([124, 134, 133, 223])
tensor([0, 0, 0, 0])
tensor([14.9721, 12.6955, 10.8439, 10.0854], grad_fn=<SliceBackward>)
24
tensor([142, 141, 159, 143])
tensor([0, 1, 1, 2])
tensor([13.2970,  9.3588,  8.0966,  1.5534], grad_fn=<SliceBackward>)
26
tensor([194, 165])
tensor([1, 1])
tensor([9.6928, 9.3516], grad_fn=<SliceBackward>)
5
tensor([124, 134, 133, 223])
tensor([0, 0, 0, 0])
tensor([14.9721, 12.6955, 10.8439, 10.0854], grad_fn=<SliceBackward>)
tensor(10.6332, grad_fn=<DivBackward0>)
[19, 7, 13, 8, 15, 12, 20, 5, 15, 14]
19
tensor([125, 194, 126, 165])
tensor([0, 0, 1, 0])
tensor([13.4145, 11.9123, 10.9482, 10.6738], grad_fn=<SliceBackward>)
7
tensor([ 74, 199,  90, 169])
tensor([0, 0, 0, 0])
tensor([13.3510, 10.9035

In [148]:
# produce a score for the label
score = 0
ind = 0
length = 4
for token_id in s_tokens[:length]:
    matched_positions = (predicted_ids == torch.Tensor([token_id])).nonzero(as_tuple=True)
    predicted_values = ctc_table.values[0][matched_positions[0], matched_positions[1]]
    # sort positions by logits
    logits, indices = torch.sort(predicted_values, descending=True)
    if matched_positions[0][indices][0] > ind:
        ind = matched_positions[0][indices][0]
        score += logits[0]

print(score/length)

tensor(13.5266, grad_fn=<DivBackward0>)


In [152]:
# produce a score for the label
score = 0
ind = 0
length = 6
for token_id in s_tokens[:length]:
    matched_positions = (predicted_ids == torch.Tensor([token_id])).nonzero(as_tuple=True)
    predicted_values = ctc_table.values[0][matched_positions[0], matched_positions[1]]
    # sort positions by logits
    logits, indices = torch.sort(predicted_values, descending=True)
    if matched_positions[0][indices][0] > ind:
        ind = matched_positions[0][indices][0]
        score += logits[0]

print(score/length)

tensor(10.6332, grad_fn=<DivBackward0>)


In [145]:
# encode negative label
n_label = 'SVETA'
n_tokens = tokenizer.tokenizer(n_label)['input_ids']
print(n_tokens)
tokenizer.tokenizer._decode(n_tokens)

[12, 25, 5, 6, 7]


'SVETA'

In [146]:
# produce a score for the label
score = 0
ind = 0
for token_id in n_tokens[:length]:
    matched_positions = (predicted_ids == torch.Tensor([token_id])).nonzero(as_tuple=True)
    predicted_values = ctc_table.values[0][matched_positions[0], matched_positions[1]]
    # sort positions by logits
    logits, indices = torch.sort(predicted_values, descending=True)
    if matched_positions[0][indices][0] > ind:
        ind = matched_positions[0][indices][0]
        score += logits[0]

print(score/length)

tensor(5.0855, grad_fn=<DivBackward0>)


In [162]:
t = torch.Tensor([1, 18, 18])
print ((t > torch.Tensor([2])).nonzero(as_tuple=True))


(tensor([1, 2]),)
